In [ ]:
!pip install --upgrade google-api-python-client
!pip3 install --upgrade oauth2client
!pip install connect
!pip install functions

In [56]:
!pip install joblib

     -------------------------------------- 298.0/298.0 kB 4.6 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import connect
import pandas as pd
import joblib

In [61]:
ids = pd.read_excel("../../data/external/Gebruikers-verkenner 20220901-20230219 rij 1-5000.xlsx", sheet_name=1, dtype="str")["Klant-ID"]
ids = pd.concat([ids, pd.read_excel("../../data/external/Gebruikers-verkenner 20220901-20230219 rij 5001-10000.xlsx", sheet_name=1, dtype="str")["Klant-ID"]]).reset_index(drop=True)

c:\Users\cosemti\.virtualenvs\ALGO-Smeg-tpBFOsly\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\cosemti\.virtualenvs\ALGO-Smeg-tpBFOsly\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [2]:
SCOPES = ['https://www.googleapis.com/auth/analytics'] #auth/analytics.readonly']
KEY_FILE_LOCATION = '../../GA_API_key.json'
VIEW_ID = '107251456'

In [3]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
service = build('analyticsreporting', 'v4', credentials=credentials)

In [52]:
data_raw = []

In [53]:
for i in range(0, len(ids)):
    id = ids[i]
    data = service.userActivity().search(
        body={
            'viewId': VIEW_ID,
            'dateRange': {
                "startDate": "2022-09-01",
                "endDate": "2023-02-19",
            },
            'user': {
                    "type": "CLIENT_ID",
                    "userId": id
                }
        }
    ).execute()

    data_raw.append({id: data})


HttpError: <HttpError 503 when requesting https://analyticsreporting.googleapis.com/v4/userActivity:search?alt=json returned "The service is currently unavailable.". Details: "The service is currently unavailable.">

In [54]:
for i in range(743, len(ids)):
    id = ids[i]
    data = service.userActivity().search(
        body={
            'viewId': VIEW_ID,
            'dateRange': {
                "startDate": "2022-09-01",
                "endDate": "2023-02-19",
            },
            'user': {
                    "type": "CLIENT_ID",
                    "userId": id
                }
        }
    ).execute()

    data_raw.append({id: data})

HttpError: <HttpError 400 when requesting https://analyticsreporting.googleapis.com/v4/userActivity:search?alt=json returned "Invalid value at 'user.user_id' (TYPE_STRING), 1140552051.1675189". Details: "[{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'user.user_id', 'description': "Invalid value at 'user.user_id' (TYPE_STRING), 1140552051.1675189"}]}]">

In [62]:
for i in range(4999, len(ids)):
    id = ids[i]
    data = service.userActivity().search(
        body={
            'viewId': VIEW_ID,
            'dateRange': {
                "startDate": "2022-09-01",
                "endDate": "2023-02-19",
            },
            'user': {
                    "type": "CLIENT_ID",
                    "userId": id
                }
        }
    ).execute()

    data_raw.append({id: data})

HttpError: <HttpError 503 when requesting https://analyticsreporting.googleapis.com/v4/userActivity:search?alt=json returned "The service is currently unavailable.". Details: "The service is currently unavailable.">

In [63]:
for i in range(8225, len(ids)):
    id = ids[i]
    data = service.userActivity().search(
        body={
            'viewId': VIEW_ID,
            'dateRange': {
                "startDate": "2022-09-01",
                "endDate": "2023-02-19",
            },
            'user': {
                    "type": "CLIENT_ID",
                    "userId": id
                }
        }
    ).execute()

    data_raw.append({id: data})

HttpError: <HttpError 503 when requesting https://analyticsreporting.googleapis.com/v4/userActivity:search?alt=json returned "The service is currently unavailable.". Details: "The service is currently unavailable.">

In [64]:
for i in range(9698, len(ids)):
    id = ids[i]
    data = service.userActivity().search(
        body={
            'viewId': VIEW_ID,
            'dateRange': {
                "startDate": "2022-09-01",
                "endDate": "2023-02-19",
            },
            'user': {
                    "type": "CLIENT_ID",
                    "userId": id
                }
        }
    ).execute()

    data_raw.append({id: data})

HttpError: <HttpError 503 when requesting https://analyticsreporting.googleapis.com/v4/userActivity:search?alt=json returned "The service is currently unavailable.". Details: "The service is currently unavailable.">

In [65]:
for i in range(9984, len(ids)):
    id = ids[i]
    data = service.userActivity().search(
        body={
            'viewId': VIEW_ID,
            'dateRange': {
                "startDate": "2022-09-01",
                "endDate": "2023-02-19",
            },
            'user': {
                    "type": "CLIENT_ID",
                    "userId": id
                }
        }
    ).execute()

    data_raw.append({id: data})

HttpError: <HttpError 429 when requesting https://analyticsreporting.googleapis.com/v4/userActivity:search?alt=json returned "Quota Error: profileId ga:107251456 has exceeded the daily request limit.". Details: "Quota Error: profileId ga:107251456 has exceeded the daily request limit.">

In [66]:
joblib.dump(data_raw, "../../data/raw/data_raw.pkl")

['../../data/raw/data_raw.pkl']

In [97]:
data_raw = joblib.load("../../data/raw/data_raw.pkl")

In [102]:
data_clean = []
for records in data_raw: 
    #test[user].pop('totalRows')
    #test[user].pop('sampleRate')

    user = list(records.keys())[0]

    for session in records[user]['sessions']: 
        for activity in session['activities']:

            result = {}
            result['id'] = user 
            result['sessionId'] = session['sessionId']
            result['deviceCategory'] = session['deviceCategory']
            result['platform'] = session['platform']
            result['sessionDate'] = session['sessionDate']
            result['dataSource'] = session['dataSource']

            for key, value in activity.items():
                if isinstance(value, dict):
                    result = {**result, **value}  # add subdict directly into the dict
                elif isinstance(value, list): 
                    for subdict in value:
                        result.update(subdict)
                else:
                    result[key] = value  # non-subdict elements are just copied

            data_clean.append(result)


In [103]:
data_clean2 = []
for records in data_clean: 
    result = {}

    for key, value in records.items():
        if isinstance(value, dict):
            result = {**result, **value}  # add subdict directly into the dict
        elif isinstance(value, list): 
            for subdict in value:
                result.update(subdict)
        else:
            result[key] = value  # non-subdict elements are just copied

    data_clean2.append(result)

In [104]:
data_preprocessed = pd.DataFrame(data_clean2)

In [106]:
data_preprocessed.to_pickle("../../data/processed/data_preprocessed.pkl")

What is the bottleneck activity? 
What transactions contain ping-pong behaviour? 
What process executions are slow? 
What are common deviations? 
How can we rearrange activities? 